<font size="5">Cross approximation in the TT format</font>

Using the torchtt.TT constructor, a TT decomposition of a given tensor can be obtained. However, in the cases where the entries of the tensor are computed using a given function, building full tensors becomes unfeasible. It is possible to 

Imports:

In [ ]:
import torch as tn
import torchtt as tntt 
import torchtt.interpolate 

We want to approximate the tensor $\mathsf{x}_{i_1...i_d}=\frac{1}{2+i_1+\cdots+i_d}$. Since the passed indices are integers of type torch.int64, casting is used.

In [ ]:
func1 = lambda I: 1/(2+tn.sum(I+1,1).to(dtype=tn.float64))

Call the torchtt.interpolate.dmrg_cross() method.

In [ ]:
N = [20]*4
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-7)

Compute the full tensor and compare to the reference.

In [ ]:
Is = tntt.meshgrid([tn.arange(0,n,dtype=tn.float64) for n in N])
x_ref = 1/(2+Is[0].full()+Is[1].full()+Is[2].full()+Is[3].full()+Is[4].full()+Is[5].full())
print('Relative error ',tn.linalg.norm(x.full()-x_ref)/tn.linalg.norm(x_ref))

We consider the case $d=10$, $n_i=32$. 

In [ ]:
N = [32]*10
x = tntt.interpolate.dmrg_cross(func1, N, eps = 1e-10)